## Web Scrapping every leads info from link.csv
#### every info is in text format, using custom keywords to separate each word and get the desired info

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:

# READ LINK IN CSV FILE 
# EXTRACT THE CODE OF EACH WEBSITE
# Store the code in the list
import csv as csv
from urllib.parse import urlparse, parse_qs

route_codes=[]

with open ('link.csv','r') as linkfile:
    reader = csv.reader(linkfile)
    i=0

    for row in reader:
        for link in row:
            parsed_url = urlparse(link)
            query_params = parse_qs(parsed_url.query)
            code = query_params.get('fldcomp_code', [''])[0]
            base = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
            # print("Code:", code)
            route_codes.append(code)
        
print(route_codes)     
len(route_codes)  

['', '5045', '11684', '15744', '21063', '19876', '17454', '20965', '10606', '17954', '16517', '6703', '20208', '22358', '13296', '20951', '20490', '20328', '22041', '14046', '20743', '10056', '19051', '18864', '20016', '20781', '22021', '13468', '20966', '7622', '20253', '11042', '19576', '21775', '22610', '20619', '19129', '17326', '21113', '19291', '14717', '11725', '16273', '20509', '17009', '17321', '5031', '22381', '4866', '22057', '19284', '19186', '19357', '16043', '17624', '19715', '10222', '21187', '22776', '20682', '20401', '20377', '9405', '4812', '16326', '20540', '20567', '17907', '21005', '22392', '20774', '11287', '21518', '14594', '20602', '18467', '4242', '16137', '20868', '6357', '17489', '16591', '5937', '16472', '20416', '21138', '21625', '18246', '11653', '22730', '22460', '5143', '6547', '22577', '22510', '20495', '20084', '15841', '14395', '22575', '19591', '5713', '22597', '22766', '16999', '14000', '19540', '19225', '18688', '10486', '12921', '22641', '15874', 

1893

In [ ]:
# c[0]='', c[1]=5045
# remove the element in the [0]
codes=[]
for c in route_codes[1:]:
    
    codes.append(c)

len(route_codes)

1893

In [105]:
# To generate random keyword or custom keyword
import random
import string
 
# initializing size of string
N = 10
 
# using random.choices()
# generating random strings
res = ''.join(random.choices(string.ascii_letters +
                             string.digits, k=N))

res2 = ''.join(random.choices(string.ascii_letters +
                             string.digits,k=N))

res3 = ''.join(random.choices(string.ascii_letters +
                             string.digits, k=N))
 
# ck is custom keyword
ck1 = str(res)
ck2 = str(res2)
ck3 = str(res3)
print(f'''The generated random string for address :  {ck1}\nThe generated random string for contact :  {ck2}''')
print ('ck3'+ ck3)

The generated random string for address :  KbkrHuW3cw
The generated random string for contact :  zhK8x23NqT
ck3qJrtUgdVuj


In [111]:
# Web Scrapping
output=[]
i=0
for code in codes:
    base_link = 'https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code='
    url = base_link + code

    page = requests.get(url)
    page = page.content
    soup = BeautifulSoup(page, 'html.parser')
   
    company_elements= soup.find_all('div', class_ = "col-sm-12")

    for comp in company_elements:
        name = comp.find('ul',class_='thumbnails').text.strip()     #name of the business company
        comp_text = comp.find('div', class_= 'col-sm-9').text
        link = comp.find('div', class_='col-sm-3').a['href']        #link of the business company
        cat_text = comp.find('div', class_='col-sm-3').text

        #-------------------------------------------------------------
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        # ---------------------------ADDRESS--------------------------
        #insert custom keyword before and after ADDRESS in the comp text
        comp_text_address_1 = re.sub(r'(\bAddress)', f'{ck1}\n\\1',comp_text)
        comp_text_address_2 = re.sub(r'(\bContact)',f'{ck2}\n\\1',comp_text_address_1)

        # find custom keyword and Remove description 
        find_ck_desc = comp_text_address_2.find(ck1)
        comp_text_address_3 = comp_text_address_2[find_ck_desc:]
        
        idx_address_1 = comp_text_address_3.index(ck1)
        idx_address_2 = comp_text_address_3.index(ck2)

        #extract the ADDRESS IN BETWEEN the custom keyword 
        block_address = comp_text_address_3[idx_address_1 + len(ck1) + 1: idx_address_2]

        #Clear double colon 
        block_address_2 = block_address.replace(ck1, '')
        block_colon_address = block_address_2.replace(':','')
        block_remove_address = block_colon_address.replace('Address','')
        
        
        #insert ck to extract address
        text_address = re.sub(r'(\bPostcode)', f'{ck1}\n\\1',block_remove_address)
        find_ck = text_address.find(ck1)

        #clean address
        clean_address = text_address[:find_ck]
        address = clean_address.replace('\n', ' ').strip()


        # ---------------------------POSTCODE--------------------------
        #insert custom keyword before and after POSTCODE in the comp text
        text_postcode = re.sub(r'(\bCity)', f'{ck2}\n\\1',text_address)

        idx_postcode_1 = text_postcode.index(ck1)
        idx_postcode_2 = text_postcode.index(ck2)

        #extract the POSTCODE IN BETWEEN the custom keyword 
        block_postcode = text_postcode[idx_postcode_1 + len(ck1) + 1: idx_postcode_2]

        #clean postcode
        postcode = block_postcode.replace('Postcode','').strip()

        # ---------------------------CITY--------------------------
        #insert custom keyword before and after CITY in the comp text
        text_city = re.sub(r'(\bState)', f'{ck3}\n\\1',text_postcode)


        idx_city_1 = text_city.index(ck2)
        idx_city_2 = text_city.index(ck3)

        #extract the CITY IN BETWEEN the custom keyword 
        block_city = text_city[idx_city_1 + len(ck2) + 1: idx_city_2]

        # Clean CITY
        city = block_city.replace('City','').strip()

        # ---------------------------STATE--------------------------
        # find ck in the textJ
        find_ck_state=text_city.find(ck3)

        # remove words before keyword
        block_state = text_city[find_ck_state:] 

        # clean STATE
        clean_ck_state= block_state.replace(ck3,'')
        state = clean_ck_state.replace('State', '').strip()
        
        #-------------------------------------------------------------
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        # ------------------------CONTACT-----------------------------
        #insert custom keyword before and after CONTACT  in the comp text
        comp_text_contact_1 = re.sub(r'(\bContact)', f'{ck1}\n\\1',comp_text)
        comp_text_contact_2 = re.sub(r'(\bFor)',f'{ck2}\n\\1', comp_text_contact_1)

        # find custom keyword and Remove description 
        find_ck_desc_contact = comp_text_contact_2.find(ck1)
        comp_text_contact_3 = comp_text_contact_2[find_ck_desc_contact:]

        idx_cont_1 = comp_text_contact_3.index(ck1)
        idx_cont_2 = comp_text_contact_3.index(ck2)

        # extract CONTACT TEXT
        block_contact = comp_text_contact_3[idx_cont_1 + len(ck1) + 1: idx_cont_2]
        
        #Clear 'no',double colon from the string
        block_colon_contact = block_contact.replace(':','')
        block_no_contact = block_colon_contact.replace('No','')

        # insert custom keyword in between line
        text_contact_1= re.sub(r'(\bPhone)',f'{ck1}\n\\1', block_no_contact)
        text_contact_2= re.sub(r'(\bFax)',f'{ck2}\n\\1', text_contact_1)
        text_contact_3= re.sub(r'(\bEmail)',f'{ck3}\n\\1', text_contact_2)

        idx_phone_1 = text_contact_3.index(ck1)
        idx_phone_2 = text_contact_3.index(ck2)
        idx_phone_3 = text_contact_3.index(ck3)

        # -----------------------------PHONE-----------------------------
        #extract the PHONE IN BETWEEN the custom keyword 
        block_phone = text_contact_3[idx_phone_1 + len(ck1) + 1: idx_phone_2]

        #clean PHONE
        phone = block_phone.replace('Phone','').strip()

        # -----------------------------FAX--------------------------------
        #extract the FAX IN BETWEEN the custom keyword 
        block_fax = text_contact_3[idx_phone_2 + len(ck2) + 1: idx_phone_3]

        #clean PHONE
        fax = block_fax.replace('Fax','').strip()

        # ----------------------------EMAIL--------------------------------
        # find custom keyword in the text
        find_ck_email= text_contact_3.find(ck3)

        #romeve words before the custom keyword 
        block_email = text_contact_3[find_ck_email:]

        #clean EMAIL
        clean_ck_email = block_email.replace(ck3,'')
        email = clean_ck_email.replace('Email Address','').strip()


        #-------------------------------------------------------------------
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        # ---------------------------CATEGORY--------------------------------
        cat_text_category_1 = re.sub(r'(Category\b)', f'{ck1}\n\\1',cat_text)
        cat_text_category_2 = re.sub(r'(\bWebsite)', f'{ck2}\n\\1',cat_text_category_1)
    

        idx_cat_1 = cat_text_category_2.index(ck1)
        idx_cat_2 = cat_text_category_2.index(ck2)


        #extract the CATEGORY IN BETWEEN the custom keyword 
        block_category=  cat_text_category_2[idx_cat_1 + len(ck1) + 1: idx_cat_2].strip()
        category = block_category.replace('Category','').strip()

        # print(category)
        # print (name)
        # print (address)
        # print (postcode)
        # print (city)
        # print (state)
        # print (phone)
        # print (fax)
        # print (email)
        # print (link)
    
        output.append([category,name,address,postcode,city,state,phone,fax,email,link,url])
        i=i+1
        print(f'Company added {i} {url}')

        break
    
    
    
print ('') 
print('------------------------------COMPLETE WEB SCRAPPING----------------------------')
   
        
    

    


Company added 1 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=5045

Company added 2 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=11684

Company added 3 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=15744

Company added 4 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=21063

Company added 5 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=19876

Company added 6 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=17454

Company added 7 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=20965

Company added 8 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=10606

Company added 9 https://www.punb.com.my/networkdirectories/index.php?route=directory/view&fldcomp_code=17954

Company ad

In [112]:
#write the list in the csv
new_df = pd.DataFrame(output, columns= ['Category','Company Name' ,'Address','Postcode','City','State','Phone No','Fax No','Email Address','Link', 'Punb Link'])
new_df.to_csv('new.csv')